### _takin' care of imports_

In [1]:
from pandas import Series, DataFrame

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [32]:
import matplotlib

In [5]:
%matplotlib inline

### _read in data_

In [6]:
weather = pd.read_table('data/daily_weather.tsv')

In [7]:
stations = pd.read_table('data/stations.tsv')

In [8]:
usage = pd.read_table('data/usage_2012.tsv')

### _repeat data fixing from step1_

In [9]:
weather['date'] = pd.to_datetime(weather['date'])

In [10]:
weather.loc[weather['season_code'] == 1, 'season_desc'] = 'winter'
weather.loc[weather['season_code'] == 2, 'season_desc'] = 'spring'
weather.loc[weather['season_code'] == 3, 'season_desc'] = 'summer'
weather.loc[weather['season_code'] == 4, 'season_desc'] = 'fall'

In [11]:
weather.loc[weather['season_desc'] == 'winter', 'season_code'] = 4
weather.loc[weather['season_desc'] == 'spring', 'season_code'] = 1
weather.loc[weather['season_desc'] == 'summer', 'season_code'] = 2
weather.loc[weather['season_desc'] == 'fall', 'season_code'] = 3

# 1. To start with, we'll need to compute the number of rentals per station per day. Use pandas to do that.

# 2a. Our stations data has a huge number of quantitative attributes: fast_food, parking, restaurant, etc... Some of them are encoded as 0 or 1 (for absence or presence), others represent counts. To start with, run a simple linear regression where the input (x) variables are all the various station attributes and the output (y) variable is the average number of rentals per day.

### _we found the average number of rentals per station in step2; recreated below_

In [18]:
usage_stations = usage[['station_start']]

In [20]:
stations_geo = DataFrame({'lat': stations.lat, 'long': stations.long})
stations_geo.index = stations.station.values

In [22]:
station_count = DataFrame(usage_stations['station_start'].value_counts())

In [24]:
stations_geo['rides'] = station_count

In [26]:
cleared = stations_geo.dropna()

# 2b. Plot the predicted values (model.predict(x)) against the actual values and see how they compare.

# 2c. In this case, there are 129 input variables and only 185 rows which means we're very likely to overfit. Look at the model coefficients and see if anything jumps out as odd.

# 2d. Go back and split the data into a training set and a test set. Train the model on the training set and evaluate it on the test set. How does it do?

# 3a. Since we have so many variables, this is a good candidate for regularization. In particular, since we'd like to eliminate a lot of them, lasso seems like a good candidate. Build a lasso model on your training data for various values of alpha. Which variables survive?

# 3b. How does this model perform on the test set?

# 4. No matter how high I make alpha, the coefficient on crossing ("number of nearby crosswalks") never goes away. Try a simple linear regression on just that variable.